In [ ]:
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

In [ ]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
IJuliaCell(vis)

In [ ]:
urdf = joinpath(Pkg.dir("MeshCatMechanisms"), "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
delete!(vis)
mvis = MechanismVisualizer(robot, urdf, vis)
set_configuration!(mvis, [1.0, -0.5])

In [ ]:
state = MechanismState(robot, randn(2), randn(2))
t, q, v = simulate(state, 5.0);

animate(mvis, t, q)

In [ ]:
using ValkyrieRobot

val = Valkyrie();
delete!(vis)
mvis = MechanismVisualizer(
    val.mechanism, 
    ValkyrieRobot.urdfpath(),
    vis, 
    package_path=[dirname(dirname(ValkyrieRobot.urdfpath()))]);

In [ ]:
using ValkyrieRobot.BipedControlUtil: Side, flipsign_if_right

function initialize!(state::MechanismState, val::Valkyrie)
    zero!(state)
    mechanism = val.mechanism
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    nothing
end
state = MechanismState(val.mechanism)
initialize!(state, val)
set_configuration!(mvis, configuration(state))